In [1]:
def editDistRecursive(x, y):
    # This implementation is very slow
    if len(x) == 0:
        return len(y)
    elif len(y) == 0:
        return len(x)
    else:
        distHor = editDistRecursive(x[:-1], y) + 1
        distVer = editDistRecursive(x, y[:-1]) + 1
        if x[-1] == y[-1]:
            distDiag = editDistRecursive(x[:-1], y[:-1])
        else:
            distDiag = editDistRecursive(x[:-1], y[:-1]) + 1
        return min(distHor, distVer, distDiag)

In [11]:
def editDistance(x, y):
    D = []
    for i in range(len(x) + 1):
        D.append([0] * (len(y) + 1))
    
    for i in range(len(x) + 1):
        D[i][0] = i
    for i in range(len(y) + 1):
        D[i][0] = i

    for i in range(1, len(x) + 1):
        for j in range(1, len(y) + 1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1

            D[i][j] = min(distHor, distVer, distDiag)
    
    # edit distance is the value in the bottom right corner of the table/bottom right cell of the array
    return D[-1][-1]

In [17]:
print([0] * 5)
[0 for _ in range(5)]

[0, 0, 0, 0, 0]


[0, 0, 0, 0, 0]

In [5]:
%%time
x = "shake spea"
y = "Shakespear"
print(editDistRecursive(x, y))

3
Wall time: 27.1 s


In [12]:
%%time
x = "shake spea"
y = "Shakespear"
print(editDistance(x, y))

3
Wall time: 0 ns


In [22]:
alphabet = ["A", "C", "G", "T"]
score = [[0, 4, 2, 4, 8], \
         [4, 0, 4, 2, 8], \
         [2, 4, 0, 4, 8], \
         [4, 2, 4, 0, 8], \
         [8, 8, 8, 8, 8]]
print(f"score: {score}")
print(f"converts from character to its offset in list alphabet: {alphabet.index('A')}")
print(f"penalty associated with A (from X) mismatching with T (from Y): {alphabet.index('G')}")
print(f"penalty associated with A (from X) mismatching with T (from Y): {score[alphabet.index('A')][alphabet.index('T')]}")
print(f"penalty associated with C (from X) being deleted in Y: {score[alphabet.index('C')][-1]}")

score: [[0, 4, 2, 4, 8], [4, 0, 4, 2, 8], [2, 4, 0, 4, 8], [4, 2, 4, 0, 8], [8, 8, 8, 8, 8]]
converts from character to its offset in list alphabet: 0
penalty associated with A (from X) mismatching with T (from Y): 2
penalty associated with A (from X) mismatching with T (from Y): 4
penalty associated with C (from X) being deleted in Y: 8


In [27]:
def globalalignment(x, y):
    D = []
    for i in range(len(x) + 1):
        D.append([0] * (len(y) + 1))
    
    # initialize first row
    for i in range(1, len(x) + 1):
        D[i][0] = D[i-1][0] + score[alphabet.index(x[i-1])][-1]
    # initalize first column
    for j in range(1, len(y) + 1):
        D[0][j] = D[0][j-1] + score[-1][alphabet.index(y[j-1])]

    for i in range(1, len(x) + 1):
        for j in range(1, len(y) + 1):
            distHor = D[i][j-1] + score[-1][alphabet.index(y[j-1])]
            distVer = D[i-1][j] + score[alphabet.index(x[i-1])][-1]
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + score[alphabet.index(x[i-1])][alphabet.index(y[j-1])]

            D[i][j] = min(distHor, distVer, distDiag)
    
    # edit distance is the value in the bottom right corner of the table/bottom right cell of the array
    return D[-1][-1]

x = "TACCAGATTCGA"
# y = "TACCAGATTCGA"
y = "TACCAAATTCGA"
globalalignment(x, y)

2

In [7]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0
    while True:
        start = a.find(b[:min_length], start)  # second argument is the index we want to start searching from
        if start == -1:
            return 0
        if b.startswith(a[start:]):
            return len(a) - start  # return the length of the overlap
        start += 1

# a = "TTACGT"
# b = "CGTGTGC"
a = "TTACGT"
b = "GTGTCG"
overlap(a, b)

0

In [11]:
from itertools import permutations

list(permutations([1,2,3], 3))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [14]:
def naive_overlap_map(reads, k):
    olaps = {}
    for a, b in permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > 0:
            olaps[(a, b)] = olen
    return olaps

# reads = ["ACGTTATC", "GATCAAGT", "TTCACGGA"]
k = 3
reads = ["ACGGATC", "GATCAAGT", "TTCACGGA"]
naive_overlap_map(reads, k)

{('ACGGATC', 'GATCAAGT'): 4, ('TTCACGGA', 'ACGGATC'): 5}